# NLP processing of the file we synced

In [1]:
# nuclio: ignore
# if the nuclio-jupyter package is not installed run !pip install nuclio-jupyter
import nuclio 

In [2]:
# package dependencies 
%nuclio cmd pip install textblob
%nuclio config spec.build.baseImage = "mlrun/mlrun"

     |████████████████████████████████| 636 kB 667 kB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 6.7 MB/s eta 0:00:01
     |████████████████████████████████| 738 kB 2.5 MB/s eta 0:00:01
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434672 sha256=697486b65bd9327d6b3578a8afd069c708c2600572a6615182fa990202cd151f
  Stored in directory: /home/jovyan/.cache/pip/wheels/ff/d5/7b/f1fb4e1e1603b2f01c2424dd60fbcc50c12ef918bafc44b155
Successfully built nltk
%nuclio: setting spec.build.baseImage to 'mlrun/mlrun'


### function code:

In [3]:
from textblob import TextBlob
import os

def nlp(text, lang='fr'):
    # process and correct the text
    blob = TextBlob(str(text))
    corrected = blob.correct()

    # debug print the text before and after correction
    print(f"Corrected text: {corrected}\noriginal: {blob}")

    # calculate sentiments
    print(f"Sentiment  polarity={corrected.sentiment.polarity}  subjectivity={corrected.sentiment.subjectivity}\n")

    # read target language from environment and return translated text
    return str(corrected.translate(to=lang))

In [4]:
def handler(context, event):
    return nlp(event.body)

In [5]:
# nuclio: end-code

### Test with file path and target language 

In [6]:
nlp('good morningg', 'es')

Corrected text: good morning
original: good morningg
Sentiment  polarity=0.7  subjectivity=0.6000000000000001



'Buenos días'

## Turn the code into a realtime micro-service (with NFS Mount)

In [7]:
import mlrun
fn = mlrun.code_to_function('my_nlp', kind="nuclio", description="nlp example")

In [10]:
fn.deploy(dashboard='http://nuclio:8070')

> 2020-11-16 01:28:34,994 [info] deploy started
[nuclio] 2020-11-16 01:29:01,898 (info) Build complete
[nuclio] 2020-11-16 01:29:02,988 Failed to deploy. Details:

Error - Container exited with status: 2
    /nuclio/pkg/dockerclient/shell.go:403

Call stack:
Container exited with status: 2
    /nuclio/pkg/dockerclient/shell.go:403
Container 408b3e9f46e3e82172b80e65f238e1ef13813926515bdc3685f3a8cb7536d80d is not healthy
    /nuclio/pkg/dockerclient/shell.go:435
Function wasn't ready in time. Logs:
20.11.16 01:29:01.779                 processor (I) Starting processor {"version": "Label: 1.5.4, Git commit: 41f4c1b4274b7ecd0b2420e01644351f4c0b007a, OS: linux, Arch: amd64, Go version: go1.14.3"}
20.11.16 01:29:01.779                 processor (D) Read configuration {"config": "{\n    \"metadata\": {\n        \"name\": \"default-my-nlp\",\n        \"namespace\": \"nuclio\",\n        \"labels\": {\n            \"mlrun/class\": \"remote\",\n            \"nuclio.io/project-name\": \"default\"\

DeployError: cannot deploy 

In [34]:
fn.invoke('', body='good evening')

b"b 'bonne soir\xc3\xa9e'"